In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 109 (delta 49), reused 94 (delta 34), pack-reused 0
Receiving objects: 100% (109/109), 252.27 KiB | 6.31 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity, sgd_optimizer
from models.model import model3

from utils import train, test, get_device, plt_fig, get_incorrect_predictions, plot_incorrect

In [4]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      transforms.Resize((28, 28)),
                                      transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                                      transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [5]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 129473368.47it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 95835200.81it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 44387552.53it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14631742.53it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# CUDA?
cuda = get_device()

# For reproducibility
torch.manual_seed(1)

if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
model3 = model3().to(cuda)

In [8]:
model_summary(model3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
         ConvBlock-5           [-1, 10, 26, 26]               0
            Conv2d-6           [-1, 20, 24, 24]           1,800
              ReLU-7           [-1, 20, 24, 24]               0
       BatchNorm2d-8           [-1, 20, 24, 24]              40
           Dropout-9           [-1, 20, 24, 24]               0
        ConvBlock-10           [-1, 20, 24, 24]               0
           Conv2d-11           [-1, 20, 22, 22]           3,600
             ReLU-12           [-1, 20, 22, 22]               0
      BatchNorm2d-13           [-1, 20, 22, 22]              40
          Dropout-14           [-1, 20,

In [9]:
from torch.optim.lr_scheduler import StepLR


optimizer = sgd_optimizer(model3, 0.03)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
EPOCHS = 15
lr = []
for epoch in range(1,EPOCHS+1):
    print("EPOCH:", epoch)
    train(model3, cuda, train_loader, optimizer, epoch)
    test(model3, cuda, test_loader)
    scheduler.step()
    lr.append(optimizer.param_groups[0]['lr'])

EPOCH: 1


Loss=0.7195419669151306 Batch_id=91 Accuracy=54.11:  20%|█▉        | 92/469 [00:10<00:44,  8.40it/s]
Exception in thread Thread-11 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 53, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 30, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 495, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", li

KeyboardInterrupt: 

In [ ]:
# Save model
torch.save(model3.state_dict(), 'best_model.pth')